# Predict TripAdvisor Rating

# ИМПОРТ БИБЛИОТЕК

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from datetime import datetime, timedelta 

import re

from itertools import combinations
from scipy.stats import ttest_ind

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# ЗАГРУЗКА ДАННЫХ

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

# ПЕРВИЧНЫЙ ОСМОТР ДАННЫХ

## Описание признаков:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.info()

In [ ]:
# Смотрим количество уникальных значений в колонках data:

for column_name in list(data.columns):
    print(column_name, len(data[column_name].unique()))

In [ ]:
# "Смотрим глазами" названия и количество уникальных значений в колонкахc некоторыми 
# категориальными признаками:
for column_name in ['City','Price Range', 'Restaurant_id']:
    print(data[column_name].value_counts())
    print('-------------------------------')

## Выводы и гипотезы/план действий

На первый взгляд наиболее значимым представляется следующие признаки, которые должны явно коррелировать с рейтингом ресторана:
* *Ranking*; 
* *Number of Reviws*.

Следующими по приоритетности являются признаки:
* *Cuisine Style*, т.к. популярность ресторана может коррелировать с популярностью представленных в нем кухонь;
* *Restaurant_id*, т.к. ряду id в данных соответствует несколько ресторанов, то, возможно этим id соответствуют брендированные сети, которые могут пользоваться большей популярностью, чем аналогичные не сетевые рестораны.
* *Price Range* т.к. отражает клиентский сегмент на который ориентирован ресторан, возможно стоит смотреть поведение другиз показателей в разрезе этого признака. 
* *Rewiews* в части дат последних отзывов, т.к. отсутствие свежих отзывов может косвенно свидетельствовать о том, что дела идут не очень хорошо, в части текстовой информации признак не интересен, т.к. два отзыва не являются сколь-либо релевантной выборкой.

Следующие признаки представляются мало интеренсыми:  
* *ID_TA*, т.к.имеет чисто технический характер и, скорей всего, не несет полезной информации;
* *URL_TA* т.к.его отработка потребует значительных усилий.

Могут представлять интерес следующие допонительные признаки:
* *Number_of_Rest* - количество ранжируемых ресторанов в каждом городе, т.к. является показателем уровня развития индустрии;
* *Population_Size* - численность населения в городе;
* *Level_of_Competition* - уровень конкуренции в городе (количество ресторанов на 1000 человек);
* *Norm_Ranking* - нормализованный для каждого города ранг ресторана;
* *Number_of_id* - количество ресторанов с одинаковым id, т.к. при прочих равных сетевые рестораны могут быть более успешными;
* *Number_оf_Cuisine* - широта ассортимента в ресторане (количество национальных кухонь), т.к. чем ширеассортимент, тем привлекательней может быть реторан;
* *Weighted_Number_of_Cuisene* - количество национальных кухонь, взвешенных по уровню их популярности;
* *Date_Rev_From_Max* - количество дней между последними отзывом во всех ресторанах и последним отзывом в данном ресторане. 

# ОБРАБОТКА ПРИЗНАКОВ И СОЗДАНИЕ НОВЫХ

## Price Range

In [ ]:
# Создаем словарь "город - самый популярный ценовой диапазон":
popular_range_dict = {}

# Функция для заполнения словаря:
def popular_price_range (city):
    
    h_number = data[(data.City == city) & (data['Price Range']== '$$$$')].ID_TA.count()
    m_number = data[(data.City == city) & (data['Price Range']== '$$ - $$$')].ID_TA.count()
    l_number = data[(data.City == city) & (data['Price Range']== '$')].ID_TA.count()
    
    if (h_number > m_number) & (m_number >= l_number):
        result = '$$$$'
    elif (l_number > m_number) & (m_number >= h_number):
        result = '$'
    else:
        result = '$$ - $$$' 
    
    return result
           

# Заполняем словарь:
for city in list(data.City.unique()):
    popular_range_dict[city] = popular_price_range (city)
    
popular_range_dict  # проверка

Как и можно было предположить, средний ценовой диапазон самый популярный во всех городах

In [ ]:
# Переводим значения Price Range в числовые признаки от 1 до 3, где 1 соответствует
# самому низкому ценовому диапазону, а 3 - самому высокому, пропуски заполняем 2 - 
# значением для среднего ценового диапазона:

# Заполняем пропуски символом '?':
data['Price Range'].fillna('$$ - $$$', inplace=True)

# Cоздаем словарь:
price_range_dict = { 
                   '$':        1,
                   '$$ - $$$': 2,
                   '$$$$':     3
                    }

# Переводим признаки в числа в новой колонке Price Range Score:
data['Price_Range_Score'] = data['Price Range'].apply(lambda x: price_range_dict[x])

display(data['Price_Range_Score'].value_counts()) #проверка

##  Number of Reviws

In [ ]:
#Смотрим минимальное значение отзывов:
min(list(data['Number of Reviews'].unique()))

In [ ]:
# Т.к.минимальное значение отзывов больше нуля, то можно заменить все пропуски на 0:
data['Number of Reviews'].fillna(0, inplace=True)

In [ ]:
# Создаем словарь со средним количеством отзывов на один ресторан в городе: 

Number_of_Reviws_Sum = data.groupby(['City'])['Number of Reviews'].sum()
Number_of_Reviws_Count = data.groupby(['City'])['Number of Reviews'].count()

med_number_of_reviws_dict = {}
for city in list(data.City.unique()):
    med_number_of_reviws_dict[city] = round(Number_of_Reviws_Sum[city]/Number_of_Reviws_Count[city])

med_number_of_reviws_dict # проверка

In [ ]:
# Заменяем все нули на среднее по городу количество отзывов:

def number_of_reviews_med(row):
    if row['Number of Reviews'] == 0:
        result = med_number_of_reviws_dict[row['City']]
    else:
        result = row['Number of Reviews']
    return result


data['Number of Reviews'] = data.apply(lambda row: number_of_reviews_med(row),axis = 1)

data['Number of Reviews'].min()  # проверка

## Cuisine Style

In [ ]:
# Меняем все пропуски на 'Typical'
data['Cuisine Style'].fillna('Typical', inplace=True)

In [ ]:
# Преобразуем строковые значения из Cuisine Style в списки:
data['Cuisine_Style_List'] = data['Cuisine Style'].str.findall(r'\w+\s*\w*\s*\w*\s*\w*\s*\w*')

display(type(data.loc[0,'Cuisine Style']))        # проверка
display(type(data.loc[0,'Cuisine_Style_List']))   # проверка

In [ ]:
# Создаем новый признак с количеством кухонь в ресторане Number оf Cuisine:
data['Number_оf_Cuisine'] = data['Cuisine_Style_List'].apply(lambda x: len(x))

data['Number_оf_Cuisine'].unique() # проверка

In [ ]:
# Среднее количество кухонь в одном ресторане:
data['Number_оf_Cuisine'].sum()/50000

In [ ]:
# На всякий случай посмотрим на выброс c 21 кухней:
list(data[data['Number_оf_Cuisine'] == 21]['Cuisine_Style_List'])

In [ ]:
# Посмотрим список всех кухонь и его размер:
all_cuisines = list(pd.Series(data['Cuisine_Style_List'].sum()).unique())
all_cuisines.sort()
display(all_cuisines)

len(all_cuisines)

In [ ]:
# Посчитаем как часто встречается та или иная кухня:
cuisines_sum = pd.Series(data['Cuisine_Style_List'].sum()).value_counts()
cuisines_sum


In [ ]:
# Посчитаем вес популярности каждой кухни как частное от деления количества упоминаний кухни  
# в датафрейме data на сумму всех полей в cuisine_sum:

# Cчитаем сумму всех полей в cuisine_sum:
cuisines_sum_total = cuisines_sum.sum()

display(cuisines_sum_total) # проверка

# Создаем справочник весов кухонь и заполняем его:
cuisines_weights = {}
for cuisen in all_cuisines:
    cuisines_weights[cuisen] = round(cuisines_sum[cuisen] / cuisines_sum_total, 5)

display(cuisines_weights)  # проверка

In [ ]:
# Добавляем в data колонку с количеством кухонь ресторана, взвешенных по их популярности:

# Создаем функцию для заполнения колонки:
def w_number_of_c_func(x):
    
    sum_weights = 0
    for c in x:
        sum_weights = sum_weights + cuisines_weights[c]
    
    return sum_weights

data['Weighted_Number_of_Cuisene'] = data.Cuisine_Style_List.apply(lambda x: w_number_of_c_func(x))

data['Weighted_Number_of_Cuisene']

## Ranking

In [ ]:
# Создаем таблицу с общим количеством ранжируемых ресторанов в городе: 
number_of_rest_data = data.groupby(['City'])['Ranking'].max()

number_of_rest_data  # проверка

In [ ]:
# Добавляем в data колонку c общим кол-вом ресторанов в городе:
data['Number_of_Rest'] = data.apply(lambda row: number_of_rest_data[row.City],axis = 1)

data['Number_of_Rest'].nunique() == 31 # проверка

In [ ]:
# Добавляем в data колонку c нормализованным рангом ресторана в городе:
data['Norm_Ranking'] = round(data.apply(lambda row: row.Ranking 
                                   / number_of_rest_data[row.City], axis = 1),2)

data['Norm_Ranking'] # проверка

In [ ]:
# Добавляем дополнительные признаки с численностью населения 
# и уровенем конкуренции в городе (количество ресторанов на 1000 жителей):

# Создаем словарь с численностью населения городов в тыс.чел.:
# (данные из Википедии 
# https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0_%D0%95%D0%B2%D1%80%D0%BE%D0%BF%D1%8B_%D1%81_%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%D0%BC_%D0%B1%D0%BE%D0%BB%D0%B5%D0%B5_500_%D1%82%D1%8B%D1%81%D1%8F%D1%87_%D1%87%D0%B5%D0%BB%D0%BE%D0%B2%D0%B5%D0%BA

population_size_dict = { 
'Amsterdam' :    873,
'Athens' :       656,
'Barcelona' :   1637,
'Berlin' :      3669,
'Bratislava' :   438,
'Brussels' :    1211,
'Budapest' :    1768,
'Copenhagen' :   616,
'Dublin' :      1173,
'Edinburgh' :    488,
'Geneva' :       496,
'Hamburg' :     1899,
'Helsinki' :     655,
'Krakow' :       779,
'Lisbon' :       507,
'Ljubljana' :    284,
'London' :      9126,
'Luxembourg' :   115,
'Lyon' :         516,
'Madrid' :      3335,
'Milan' :       1404,
'Munich' :      1472,
'Oporto' :       268,
'Oslo' :         673,
'Paris' :       2148,
'Prague' :      1324,
'Rome' :        2791,
'Stockholm' :    962,
'Vienna' :      1921,
'Warsaw' :      1794,
'Zurich' :       429
}

# Создаем колонку с численностью населения в городе:
data['Population_Size'] = data.apply(lambda row: population_size_dict[row.City],axis = 1)

display(data['Population_Size']) # проверка

# Создаем колонку с уровнем конкуренции в городе:
data['Level_of_Competition'] = round(data.apply(lambda row: number_of_rest_data[row.City] 
                                 / population_size_dict[row.City], axis = 1),1)

display(data['Level_of_Competition']) # проверка

## Restaurant_id

In [ ]:
# Создаем таблицу с общим количеством ресторанов с одинаковым id: 
number_of_rest_id = data.groupby(['Restaurant_id'])['Ranking'].count()

number_of_rest_id.sum() == 50000  # проверка

In [ ]:
# Добавляем в data колонку c общим кол-вом ресторанов c одинаковым рeйтингом:
data['Number_of_id'] = data.apply(lambda row: number_of_rest_id[row.Restaurant_id],axis = 1)

data['Number_of_id'].nunique() == number_of_rest_id.nunique() # проверка

## Reviews

In [ ]:
# Посмотрим на данные:
data.Reviews.value_counts()

In [ ]:
# Запоняем пропуски самым популярным значением:
data.Reviews.fillna('[[],[]]', inplace = True)

In [ ]:
# Создаем столбец Review_Dates содержащий список дат отзывов:
pattern = re.compile('\d+\/\d+\/\d+')
data['Review_Date'] = data.Reviews.apply(pattern.findall)

data['Review_Date'].sample(5) # проверка

In [ ]:
# Смотрим сколько всего бывает дат:

data['Number_of_Date'] = data.Review_Date.apply(lambda x: len(list(x)))
data.Number_of_Date.value_counts()

Более 2-х дат в записи свидельствует о том, что в поле попали даты, упоминаемые в комментариях

In [ ]:
# Убираем даты упоминаемые из комментариях отзывов:
data.Review_Date = data.Review_Date.apply(lambda x: [x[-2], x[-1]] if len(x) > 2 else x)

# Проверяем, что получилось:
data['Number_of_Date'] = data.Review_Date.apply(lambda x: len(list(x))) 
data.Number_of_Date.value_counts() 

In [ ]:
# Разносим даты по 2-м новым столбцам, так чтобы в столбце Date_Rev_1 была бОльшая дата
# и переводим их в формат datetime64:

# Функции для заполнения столбцов:

def date_rev_1_func(x):
    if len(x) == 0:
        result = None
    elif len(x) == 1:
        result = x[0]
    else: 
        result = max(x)
    return result

def date_rev_2_func(x):
    if len(x) == 0:
        result = None
    elif len(x) == 1:
        result = x[0]
    else:
        result = min(x)
    return result

# Заполняем столбцы:

data['Date_Rev_1'] = pd.to_datetime(data.Review_Date.apply(lambda x: date_rev_1_func(x)))

display(data['Date_Rev_1'].sample(3)) # проверка

data['Date_Rev_2'] = pd.to_datetime(data.Review_Date.apply(lambda x: date_rev_2_func(x)))

display(data['Date_Rev_2'].sample(3)) # проверка

In [ ]:
# Cмотрим разницу между датами 2-х отзывов и ее макcимальное значение:
data['Date_Rev_delta'] = (data.Date_Rev_1 - data.Date_Rev_2)/ np.timedelta64(1, "D")

x = list(data['Date_Rev_delta'].unique()) 
x.sort()
print(x[-1])

In [ ]:
# Создаем колонку с количеством дней, отделяющих последний отзыв от date_max:

# Находим максимальную дату отзыва в датасете:
date_max = data['Date_Rev_1'].max()
display(date_max)

# Заполняем колонку:
data['Date_Rev_From_Max'] = data.apply(lambda row: None if len(row.Review_Date) == 0  
                                       else ((date_max-row.Date_Rev_1)), axis=1) / np.timedelta64(1, "D")  

# Находим максимальное значение в колонке:
bad_date = data['Date_Rev_From_Max'].max()

# Заполняем пропуски максимальным значением в колонке:
data.Date_Rev_From_Max.fillna(bad_date, inplace = True)

data.Date_Rev_From_Max.value_counts() # проверка

## City

In [ ]:
# Проводим One-Hot Encoding:

data = pd.get_dummies(data, columns=['City'], dummy_na=True)

data.info() # проверка

## Убираем технические колонки и смотрим, что получилось

In [ ]:
data.drop(['Cuisine_Style_List','Review_Date', 'Number_of_Date', 
           'Date_Rev_1', 'Date_Rev_2', 'Date_Rev_delta'], inplace=True, axis=1)

data.info()

# АНАЛИЗ ДАННЫХ 

## Распределение признаков

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

## Распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

## Корреляция числовых переменных

In [ ]:
# Сфорсмруем список признаков, которые исключаем из корреляционного анализа
cols_to_drop = ['sample', 'City_Amsterdam',  'City_Athens',  'City_Barcelona', 'City_Berlin',  
                'City_Bratislava',  'City_Brussels',  'City_Budapest',  'City_Copenhagen',  
                'City_Dublin',  'City_Edinburgh',  'City_Geneva',  'City_Hamburg',  
                'City_Helsinki', 'City_Krakow',  'City_Lisbon',  'City_Ljubljana',  
                'City_London',  'City_Luxembourg',  'City_Lyon',  'City_Madrid',  'City_Milan',  
                'City_Munich',  'City_Oporto',  'City_Oslo',  'City_Paris', 'City_Prague',  
                'City_Rome',  'City_Stockholm',  'City_Vienna',  'City_Warsaw',  'City_Zurich',  
                'City_nan']
                

In [ ]:
plt.rcParams['figure.figsize'] = (15,15)
data_corr = data
sns.heatmap(data_corr[data_corr['sample']== 1].drop(cols_to_drop, axis=1).corr(), square=True,
            annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu");
plt.tight_layout()

С целевым рейтингом больше всего коррелируют нормированный и обычный рэнкинги. 
Показатели Number_of_id, Number_of_Cusine и Population_Size имеют стльную корреляцию с другими признаками, поэтому их надо убрать.

## Тест Тьюринга для номинативных перемееных

In [ ]:
# Создадим функцию для проведения теста Стьюдента:

def get_stat_dif_2(column):
    
    cols = data[data['sample'] == 1].loc[:, column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))
    # Тест проводим на изначальном наборе данных без NA значений для целевого столбца, 
    # столбца с признаком, дополнительно исключив 0 для оценок
    stud_stat = data[data['sample'] == 1]
    for comb in combinations_all:
        if ttest_ind(stud_stat.loc[data[data['sample'] == 1].loc[:, column] == comb[0], 'Rating'],
                     stud_stat.loc[data[data['sample'] == 1].loc[:, column] == comb[1], 'Rating']).pvalue <= 0.05/len(combinations_all):  # учли поправку Бонферони
            # print('Найдены статистически значимые различия для колонки', column)
            pass
        else:
            return column 
            break

In [ ]:
# Сщздаем сет статистически незначимых признаков:

to_remove_features = set()

# Проходим по колонкам, которые исключали из корреляционного анализа
for column in cols_to_drop: 
    to_remove_features.add(get_stat_dif_2(column))
    
print('\n Список признаков на удаление из обучения модели:', to_remove_features)

# ОБРАБОТКА ДАННЫХ

In [ ]:
# df_preproc = preproc_data(data)
df_preproc = data

In [ ]:
# Убираем ненужные признаки:

# Убираем коллинераные признаки:
df_preproc.drop(['Number_of_id','Population_Size','Number_оf_Cuisine'], axis = 1, inplace=True)

# Убираем статиcnически незначимые по тесту Стьюдента признаки:
df_preproc.drop(['City_Helsinki', 'City_Hamburg', 'City_Bratislava', 'City_Geneva', 'City_Zurich', 
                 'City_Copenhagen', 'City_Luxembourg'], axis = 1, inplace=True)


# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in df_preproc.columns if df_preproc[s].dtypes == 'object']
df_preproc.drop(object_columns, axis = 1, inplace=True)

df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Так как признак рейтинга имеет шаг 0.5, округляем предсказание.
y_pred = np.round(y_pred * 2) / 2

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
# Так как признак рейтинга имеет шаг 0.5, округляем предсказание.
predict_submission = np.round(predict_submission * 2) / 2

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)